In [8]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig
import matplotlib.patches as patches
import cv2
import pandas as pd
import sys
import poissonimageediting as poisson

np.set_printoptions(threshold=sys.maxsize)

def rotate(image, angle, center=None, scale=1.0):
    (h, w) = image.shape[:2]
    if center is None:
        center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def retrieve_ellipse(src, u, v, A, o):
    image = src.copy()
    height, width = image.shape[:2]
    vals, vecs = eig(A)
    major_axis_length = 1 / np.sqrt(vals[0])
    minor_axis_length = 1 / np.sqrt(vals[1])
    angle = np.arctan2(vecs[1, 0], vecs[0, 0])

    y, x = np.ogrid[:height, :width]
    x = x - u
    y = y - v

    ellipse_mask = ((x * np.cos(angle) + y * np.sin(angle))**2 / major_axis_length**2 +
                    (y * np.cos(angle) - x * np.sin(angle))**2 / minor_axis_length**2) <= 1
    
    not_rotate_image = np.zeros_like(image)
    not_rotate_image[ellipse_mask] = image[ellipse_mask]

    image = rotate(image, o - 180, center=(u, v))

    em_int = np.where(ellipse_mask, 255, 0).astype(np.uint8)
    em_int = np.dstack([em_int, em_int, em_int])
    em_int = rotate(em_int, o - 180, center=(u, v))
    em_int = em_int[:, :, 0]
    ellipse_mask = np.where(em_int == 255, True, False)

    indices = np.where(ellipse_mask)
    min_y, max_y = indices[0].min(), indices[0].max()
    min_x, max_x = indices[1].min(), indices[1].max()

    selected_region = np.zeros_like(image)
    selected_region[ellipse_mask] = image[ellipse_mask]

    return selected_region[min_y:max_y + 1, min_x:max_x + 1], ellipse_mask[min_y:max_y + 1, min_x:max_x + 1], not_rotate_image

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    rotated = cv2.warpAffine(image, M, (new_w, new_h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    return rotated

def paste_image_with_alpha(bg_img, overlay_img, center_x, center_y):
    H, W, _ = bg_img.shape
    h, w, _ = overlay_img.shape
    start_x = max(center_x - w // 2, 0)
    start_y = max(center_y - h // 2, 0)
    overlay_start_x = max(0, w // 2 - center_x)
    overlay_start_y = max(0, h // 2 - center_y)
    overlay_end_x = min(w, W - center_x + w // 2)
    overlay_end_y = min(h, H - center_y + h // 2)
    bg_end_x = start_x + overlay_end_x - overlay_start_x
    bg_end_y = start_y + overlay_end_y - overlay_start_y
    alpha_overlay = overlay_img[overlay_start_y:overlay_end_y, overlay_start_x:overlay_end_x, 3] / 255.0
    
    for c in range(3):
        bg_img[start_y:bg_end_y, start_x:bg_end_x, c] = np.where(alpha_overlay > 0, overlay_img[overlay_start_y:overlay_end_y, overlay_start_x:overlay_end_x, c], bg_img[start_y:bg_end_y, start_x:bg_end_x, c])
    
    return bg_img

def add_alpha_channel(image):
    b, g, r = cv2.split(image)
    alpha = np.ones(b.shape, dtype=b.dtype) * 255
    return cv2.merge((b, g, r, alpha))

# Load images and data
image = cv2.imread('test_imgs/01.png').astype(np.uint8)
dataset_images_dir = "data/holiday_images"
target_df = pd.read_pickle('data/target_df.pkl')
dataset_df = pd.read_pickle('data/holiday_df.pkl')

matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
target_descriptors = np.array(target_df['descriptor'].tolist(), dtype=np.float32)
dataset_descriptors = np.array(dataset_df['descriptor'].tolist(), dtype=np.float32)
matches = matcher.match(target_descriptors, dataset_descriptors)
target_df['match_idx'] = np.array([match.trainIdx for match in matches])
target_df = target_df.sort_values(by=['size'], ascending=[False])

In [20]:
target_df

,image_name,coordinate,scale,orientation,matrix,eigenvalues,eigenvectors,semi_major_axis,semi_minor_axis,size,angle,descriptor
0,01.png,"(265.326, 11.8137)",18.323856,-178.632952,"[[0.00299271, -0.000604459], [-0.000604459, 0....","[0.002978285184905387, 0.02832202481509461]","[[-0.9997153760964067, 0.023857216862405706], ...",18.323856,5.942071,342.061805,-3.117733,"[0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 126.0, 0.0, 0...."
1,01.png,"(298.968, 11.7613)",15.816728,-176.458907,"[[0.00406315, -0.00106412], [-0.00106412, 0.02...","[0.003997299539765912, 0.02125895046023409]","[[-0.9980907567045683, 0.061764402214400146], ...",15.816728,6.858499,340.796885,-3.079789,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 46.0, 0.0, 0.0,..."
2,01.png,"(307.376, 11.685)",14.616900,178.418101,"[[0.00469393, 0.000487391], [0.000487391, 0.02...","[0.004680470032437781, 0.02234255996756222]","[[-0.999618886185318, -0.02760583962903752], [...",14.616900,6.690114,307.212351,3.113983,"[0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 79.0, 18.0, 0.0..."
3,01.png,"(314.48, 12.4368)",17.254270,170.566924,"[[0.00379769, 0.0026406], [0.0026406, 0.0192526]]","[0.0033589752040047577, 0.019691314795995247]","[[-0.9864777119362055, -0.16389546623719878], ...",17.254270,7.126276,386.286131,2.976954,"[16.0, 1.0, 0.0, 0.0, 11.0, 18.0, 93.0, 93.0, ..."
4,01.png,"(548.047, 12.0133)",17.391688,-179.892504,"[[0.00330615, -2.44128e-05], [-2.44128e-05, 0....","[0.0033061041976317363, 0.016318245802368268]","[[-0.9999982400126757, 0.0018761587222050893],...",17.391688,7.828224,427.715382,-3.139716,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 122.0, 0.0, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
767,01.png,"(560.767, 361.151)",185.070659,175.556743,"[[3.07923e-05, 2.05417e-05], [2.05417e-05, 0.0...","[2.919610096367703e-05, 0.000295146199036323]","[[-0.9969945465167589, -0.0774717639907755], [...",185.070659,58.207829,33842.999456,3.064043,"[28.0, 43.0, 34.0, 27.0, 16.0, 12.0, 7.0, 4.0,..."
768,01.png,"(317.425, 367.922)",119.924266,50.619730,"[[9.61867e-05, -2.18789e-05], [-2.18789e-05, 8...","[6.953218284910574e-05, 0.00011414561715089426]","[[0.634464380565544, 0.772952100581647], [0.77...",119.924266,93.598821,35263.654730,0.883481,"[2.0, 1.0, 13.0, 40.0, 15.0, 10.0, 31.0, 77.0,..."
769,01.png,"(100.711, 374.921)",127.284183,90.227400,"[[0.000215226, 6.09236e-07], [6.09236e-07, 6.1...","[6.172358200223463e-05, 0.00021522841799776534]","[[-0.003968870390483995, 0.999992124002896], [...",127.284183,68.163235,27256.777261,1.574765,"[29.0, 14.0, 7.0, 24.0, 18.0, 1.0, 3.0, 64.0, ..."
770,01.png,"(575.02, 409.199)",168.072980,177.581045,"[[3.56435e-05, 5.76235e-06], [5.76235e-06, 0.0...","[3.540007630402613e-05, 0.00017205042369597388]","[[-0.9991089221370543, -0.042206180899645805],...",168.072980,76.238111,40255.006434,3.099374,"[47.0, 12.0, 10.0, 0.0, 2.0, 40.0, 63.0, 101.0..."


In [22]:
target_path = 'data/01.png'
dataset_images_dir = "data/holiday_images"
target_df = pd.read_pickle('data/target_df.pkl')
dataset_df = pd.read_pickle('data/holiday_df.pkl')

image = cv2.imread('test_imgs/01.png').astype(np.uint8)

matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
target_descriptors = np.array(target_df['descriptor'].tolist(), dtype=np.float32)
dataset_descriptors = np.array(dataset_df['descriptor'].tolist(), dtype=np.float32)
matches = matcher.match(target_descriptors, dataset_descriptors)
target_df['match_idx'] = np.array([match.trainIdx for match in matches])
target_df = target_df.sort_values(by=['size'], ascending=[False])

def get_ellipse(path, u, v, angle, major_axis_length, minor_axis_length):
    img = cv2.imread(path).astype(np.float32)
    height, width = img.shape[: 2]

    y, x = np.ogrid[:height, :width]
    x = x - u
    y = y - v

    ellipse_mask = ((x * np.cos(angle) + y * np.sin(angle))**2 / major_axis_length**2 + (y * np.cos(angle) - x * np.sin(angle))**2 / minor_axis_length**2) <= 1
    
    ellipse_img = np.zeros_like(image)
    ellipse_img[ellipse_mask] = image[ellipse_mask]

    return ellipse_img


for target_idx, target_row in target_df.iterrows():
    dataset_idx = target_row['match_idx']
    dataset_row = dataset_df.loc[dataset_idx]

    # target_ellipse_img = get_ellipse(target_path, target_row['coordinate'][0], target_row['coordinate'][1], target_row['angle'], target_row['semi_major_axis'], target_row['semi_minor_axis'])
    dataset_elipse_img = get_ellipse(f"{dataset_images_dir}/{dataset_row['image_name']}", dataset_row['coordinate'][0], dataset_row['coordinate'][1], dataset_row['angle'], dataset_row['semi_major_axis'], dataset_row['semi_minor_axis'])


    if target_idx == 771: break


image_name                                                    01.png
coordinate                                        (575.417, 419.393)
scale                                                      192.07462
orientation                                              -170.069473
matrix             [[2.79806e-05, -4.99744e-06], [-4.99744e-06, 5...
eigenvalues         [2.7105663086150243e-05, 5.6524836913849755e-05]
eigenvectors       [[-0.9850175819893834, 0.17245394507458628], [...
semi_major_axis                                            192.07462
semi_minor_axis                                           133.008789
size                                                    80260.192263
angle                                                      -2.968272
descriptor         [15.0, 55.0, 55.0, 9.0, 17.0, 12.0, 19.0, 24.0...
match_idx                                                        190
Name: 771, dtype: object
image_name                                                    01.png
coordinat